# Comparison of AuxTel ISR before and after sequencer/voltage change
Craig Lage 05-Jan-23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from lsst.daf.butler import Butler
from lsst.summit.utils import BestEffortIsr
from lsst.pipe.tasks.quickFrameMeasurement import QuickFrameMeasurementTask
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

In [ ]:
def plotExp(exp):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure


In [ ]:
expIds = [2022060900930, 2022121200752]
butler_before = Butler('/repo/main', collections=["LATISS/raw/all", "LATISS/calib"])
butler_after = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib"])
best_effort_isr_before = BestEffortIsr(embargo=False)
best_effort_isr_after = BestEffortIsr(embargo=True)

 ### An exposure from before the change

In [ ]:
dataId = {'exposure': expIds[0], 'detector': 0, "instrument": 'LATISS'}
best_effort_isr_before.doWrite = False  # Don't write to butler database
exp_before = best_effort_isr_before.getExposure(dataId, skipCosmics=False)
bias_before = butler_before.get('bias', dataId=dataId)
raw_before = butler_before.get('raw', dataId=dataId)

In [ ]:
fig = plotExp(exp_before)
plt.suptitle(f"{expIds[0]}")
fig.savefig(f"/home/c/cslage/u/AuxTel/isr/before_after/Before.png")

 ### An exposure from after the change

In [ ]:
dataId = {'exposure': expIds[1], 'detector': 0, "instrument": 'LATISS'}
best_effort_isr_after.doWrite = False  # Don't write to butler database
exp_after = best_effort_isr_after.getExposure(dataId, skipCosmics=False)
bias_after = butler_after.get('bias', dataId=dataId)
raw_after = butler_after.get('raw', dataId=dataId)

In [ ]:
fig = plotExp(exp_after)
plt.suptitle(f"{expIds[1]}")
fig.savefig(f"/home/c/cslage/u/AuxTel/isr/before_after/After.png")

 ### Looking at the wave and the segment boundaries

In [ ]:
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle(f"AuxTel bias stability - Amp C10")
names = ['V25', 'V26']
for i, [exp, bias] in enumerate([[exp_before, bias_before], [exp_after, bias_after]]):
    expPlot = np.median(exp.image.array[2100:3900, :], axis=0)
    expPlot -= np.mean(expPlot)
    biasPlot = np.median(bias.image.array[2100:3900, :], axis=0)
    biasPlot -= np.mean(biasPlot)
    plt.subplot(2,2,2*i+1)
    plt.title(f"{expIds[i]} - {names[i]}")
    plt.plot(expPlot, label="Exposure")
    #plt.plot(biasPlot, label="Master bias")
    plt.xlim(0, 400)
    plt.ylim(-5,5)
    #plt.legend()
    plt.xlabel("X coord (pixels)")
    plt.ylabel("ADU")

    plt.subplot(2,2,2*i+2)
    plt.title(f"{expIds[i]} - {names[i]}")
    plt.plot(expPlot, label="Exposure")
    #plt.plot(biasPlot, label="Master bias")
    plt.xlim(450, 520)
    plt.ylim(-40,40)
    #plt.legend()
    plt.xlabel("X coord (pixels)")
    plt.ylabel("ADU")

fig.savefig("/home/c/cslage/u/AuxTel/isr/before_after/Bias_Stability.pdf")

 ### More detailed look at the segment boundaries

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle(f"AuxTel bias stability - Amp C10")
names = ['V25', 'V26']
for i, [raw, bias] in enumerate([[raw_before, bias_before], [raw_after, bias_after]]):
    isrResult = isrTask.run(raw)
    rawPlot = np.median(isrResult.exposure.image.array[2100:3900, 0:520], axis=0)
    rawPlot -= np.mean(rawPlot)
    biasPlot = np.median(bias.image.array[2100:3900, 0:520], axis=0)
    biasPlot -= np.mean(biasPlot)
    plt.subplot(1,2,i+1)
    plt.title(f"{expIds[i]} - {names[i]}")
    plt.plot(rawPlot, label="Raw")
    plt.plot(biasPlot, ls = '--', label="MasterBias")
    plt.xlim(490, 511)
    #plt.ylim(-40,40)
    plt.legend()
    plt.xlabel("X coord (pixels)")
    plt.ylabel("ADU")

fig.savefig("/home/c/cslage/u/AuxTel/isr/before_after/Bias_Stability_2.pdf")